In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import keras
from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.models import Sequential, load_model, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import optuna

import psycopg2
from config import *

import utm
import time

from sklearn.ensemble import RandomForestClassifier
import optuna

In [ ]:
def get_closest(lon,lat,estaciones):
    distancias_list=[]
    cercano=0
    closest=10000000
    for i in range(len(estaciones)):
        dist=(estaciones['longitude'].iloc[i]-lon)**2+(estaciones['latitude'].iloc[i]-lat)**2
        distancias_list.append(dist)
        if(dist<closest):
            closest=dist
            cercano=estaciones['nombrecorto'].iloc[i]
    estaciones['distancias']=distancias_list
    return cercano

In [ ]:
conexion = psycopg2.connect(database=db_database, 
                                user=db_user, 
                                password=db_password, 
                                host=db_host, 
                                port=db_port)

In [ ]:
query='''select date, variedad, min(phenologystageid) as phenologystageid, codigocatastro as codigo 
from redfara.redfara_fenologia
where especie='VIÑEDO VINIFICACION'
group by date, variedad, codigo, phenologystageid;'''
phenological_data = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
phenological_data

In [ ]:
query='select * from cadastral.parcelas where codigo in '+ str(list(phenological_data.codigo.unique())).replace('[','(').replace(']',')') + ';'
cadastral_data = pd.read_sql_query(query, con=conexion)
conexion.commit()


In [ ]:
cadastral_data['longitude']=cadastral_data['coordenadas_epsgwgs84'].apply(lambda x:float(x[0]))
cadastral_data['latitude']=cadastral_data['coordenadas_epsgwgs84'].apply(lambda x:float(x[1]))
cadastral_data=cadastral_data[['codigo', 'longitude','latitude','altitud']].drop_duplicates()
cadastral_data

In [ ]:
query='select * from public.estara;'
stations = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
stations

In [ ]:
stations['longitude']=stations.longitud.str.split("'|º").apply(lambda x:int(x[0])+int(x[1])/60+int(x[2])/3600000)
stations['longitude']=stations['longitude']*(stations.longitud.str.contains('E')*2-1)
stations['latitude']=stations.latitud.str.split("'|º").apply(lambda x:int(x[0])+int(x[1])/60+int(x[2])/3600000)
stations['latitude']=stations['latitude']*(stations.latitud.str.contains('N')*2-1)
stations=stations[['nombrecorto','longitude','latitude', 'altitud']]
stations

In [ ]:
cercanias=[]
for i in range(len(cadastral_data)):
    cercanias.append(get_closest(cadastral_data.iloc[i].longitude,cadastral_data.iloc[i].latitude,stations))
cadastral_data['closest']=cercanias
cadastral_data

In [ ]:
closest_stations = stations[stations.nombrecorto.isin(cadastral_data.closest.unique())]
closest_stations

In [ ]:
lowest_year=phenological_data.date.dt.year.min()
lowest_year

In [ ]:
query='select * from public.meteorological_data WHERE anio >= ' + str(lowest_year) + ' AND estacion in ' + str(list(closest_stations.nombrecorto.unique())).replace('[','(').replace(']',')') + ';'
meteorological_data = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
meteorological_data = meteorological_data.drop(['ubi', 'season', 'hourFrac_sum'], axis=1)
variables_diarias_min=['tmed_min', 'rad_min']
variables_diarias_max=['tmed_max', 'rad_max']
variables_diarias_mean=['tmed_mean', 'rad_mean', 'wind_N', 'wind_NE', 'wind_E','wind_SE', 'wind_S', 'wind_SW', 
                        'wind_W', 'wind_NW']
variables_semanales=['gdd_4.5_t0_Tbase_sum',
       'gdd_4.5_t0_TbaseMax_sum', 'gdd_4.5_1_Tbase_sum',
       'gdd_4.5_1_TbaseMax_sum', 'gdd_4.5_2_Tbase_sum',
       'gdd_4.5_2_TbaseMax_sum', 'gdd_10.0_t0_Tbase_sum',
       'gdd_10.0_t0_TbaseMax_sum', 'gdd_10.0_1_Tbase_sum',
       'gdd_10.0_1_TbaseMax_sum', 'gdd_10.0_2_Tbase_sum',
       'gdd_10.0_2_TbaseMax_sum', 'chillingDD_7.0_t0_Tbase_sum',
       'chillingDD_7.0_t0_Tbasemin_sum', 'chillingDD_7.0_t0_Utah_sum',
       'chillingDD_7.0_1_Tbase_sum', 'chillingDD_7.0_1_Tbasemin_sum',
       'chillingDD_7.0_1_Utah_sum', 'chillingDD_7.0_2_Tbase_sum',
       'chillingDD_7.0_2_Tbasemin_sum', 'chillingDD_7.0_2_Utah_sum', 'rad_sum',
       'precip_sum', 'winkler_4.5_Tbase', 'winkler_4.5_TbaseMax',
       'winkler_10.0_Tbase', 'winkler_10.0_TbaseMax',
       'gdd_4.5_t0_Tbase_sum_Cumm', 'gdd_4.5_t0_TbaseMax_sum_Cumm',
       'gdd_4.5_1_Tbase_sum_Cumm', 'gdd_4.5_1_TbaseMax_sum_Cumm',
       'gdd_4.5_2_Tbase_sum_Cumm', 'gdd_4.5_2_TbaseMax_sum_Cumm',
       'gdd_10.0_t0_Tbase_sum_Cumm', 'gdd_10.0_t0_TbaseMax_sum_Cumm',
       'gdd_10.0_1_Tbase_sum_Cumm', 'gdd_10.0_1_TbaseMax_sum_Cumm',
       'gdd_10.0_2_Tbase_sum_Cumm', 'gdd_10.0_2_TbaseMax_sum_Cumm',
       'chillingDD_7.0_t0_Tbase_sum_Cumm',
       'chillingDD_7.0_t0_Tbasemin_sum_Cumm',
       'chillingDD_7.0_t0_Utah_sum_Cumm', 'chillingDD_7.0_1_Tbase_sum_Cumm',
       'chillingDD_7.0_1_Tbasemin_sum_Cumm', 'chillingDD_7.0_1_Utah_sum_Cumm',
       'chillingDD_7.0_2_Tbase_sum_Cumm', 'chillingDD_7.0_2_Tbasemin_sum_Cumm',
       'chillingDD_7.0_2_Utah_sum_Cumm', 'rad__t0__Cumm', 'rad__1__Cumm',
       'rad__2__Cumm', 'precip__t0__Cumm', 'precip__1__Cumm',
       'precip__2__Cumm', 'winkler_4.5_t0_Tbase_Cumm',
       'winkler_4.5_t0_TbaseMax_Cumm', 'winkler_4.5_1_Tbase_Cumm',
       'winkler_4.5_1_TbaseMax_Cumm', 'winkler_4.5_2_Tbase_Cumm',
       'winkler_4.5_2_TbaseMax_Cumm', 'winkler_10.0_t0_Tbase_Cumm',
       'winkler_10.0_t0_TbaseMax_Cumm', 'winkler_10.0_1_Tbase_Cumm',
       'winkler_10.0_1_TbaseMax_Cumm', 'winkler_10.0_2_Tbase_Cumm',
       'winkler_10.0_2_TbaseMax_Cumm']
meteorological_data

In [ ]:
meteorological_data.columns

In [1]:
datos_meteo_buenos_list=[]
n_dias_atras=14
n_dias_alante=5
for estacion in meteorological_data['estacion'].unique():
    datos_est=meteorological_data[meteorological_data.estacion==estacion].sort_values('fecha').set_index('fecha')
    datos_meteo_buenos_est_list=[datos_est[['estacion']]]
    for var in variables_diarias_min:
        datos_var_est=datos_est[[var]].resample('1D').min()
        for i in range(1,n_dias_atras):
            datos_var_est[var + " " + str(i) + "_dias_atras"]=datos_var_est[var].resample('1D').min().shift(i)
        for i in range(1,n_dias_alante):
            datos_var_est[var + " " + str(i) + "_dias_adelante"]=datos_var_est[var].resample('1D').min().shift(-i)
        datos_meteo_buenos_est_list.append(datos_var_est)
    for var in variables_diarias_max:
        datos_var_est=datos_est[[var]].resample('1D').max()
        for i in range(1,n_dias_atras):
            datos_var_est[var + " " + str(i) + "_dias_atras"]=datos_var_est[var].resample('1D').max().shift(i)
        for i in range(1,n_dias_alante):
            datos_var_est[var + " " + str(i) + "_dias_adelante"]=datos_var_est[var].resample('1D').max().shift(-i)
        datos_meteo_buenos_est_list.append(datos_var_est)
    for var in variables_diarias_mean:
        datos_var_est=datos_est[[var]].resample('1D').max()
        for i in range(1,n_dias_atras):
            datos_var_est[var + " " + str(i) + "_dias_atras"]=datos_var_est[var].resample('1D').max().shift(i)
        for i in range(1,n_dias_alante):
            datos_var_est[var + " " + str(i) + "_dias_adelante"]=datos_var_est[var].resample('1D').max().shift(-i)
        datos_meteo_buenos_est_list.append(datos_var_est)
    for var in variables_semanales:
        datos_var_est=datos_est[[var]].resample('1D').max()
        for i in range(1,1+n_dias_atras//7):
            datos_var_est[var + " " + str(i) + "_semanas_atras"]=datos_var_est[var].resample('1D').ffill().shift(i*7)
        for i in range(1,1+n_dias_alante//7):
            datos_var_est[var + " " + str(i) + "_semanas_adelante"]=datos_var_est[var].resample('1D').ffill().shift(-i*7)
        datos_meteo_buenos_est_list.append(datos_var_est)
        
    datos_var_est=pd.concat(datos_meteo_buenos_est_list,axis=1).reset_index()
    datos_meteo_buenos_list.append(datos_var_est)
datos_meteo_buenos=pd.concat(datos_meteo_buenos_list).dropna()
datos_meteo_buenos

NameError: name 'meteorological_data' is not defined

In [ ]:
datos_part=pd.merge(phenological_data.reset_index(), cadastral_data, left_on='codigo', right_on='codigo', how='outer', indicator=True)
datos_part

In [ ]:
datos_part2=datos_part[datos_part._merge=='both']
datos_part2

In [ ]:
datos_campos=[]
for campo in datos_part2.codigo.unique():
    datos_part3=datos_part2[datos_part2.codigo==campo]
    datos_campos.append(datos_part3.sort_values('date').set_index('date').resample('1D').ffill().dropna())
    
datos_part=pd.concat(datos_campos).reset_index()
datos_part

In [ ]:
datos_part['anio']=datos_part['date'].dt.year
datos_part['dia']=datos_part['date'].dt.dayofyear
datos_part=datos_part.drop(columns=['index','_merge','date'])
datos_part=pd.get_dummies(datos_part,columns=['variedad'])

In [ ]:
datos_meteo_buenos['anio']=datos_meteo_buenos.fecha.dt.year
datos_meteo_buenos['dia']=datos_meteo_buenos.fecha.dt.dayofyear
datos_total=pd.merge(datos_part, datos_meteo_buenos, left_on=['closest','anio','dia'], right_on=['estacion','anio','dia'])
datos_total=datos_total.drop(['closest','fecha','estacion'],axis=1)
datos_total

In [ ]:
query='''select codigo, date, AVG(min) as min, AVG(max) as max, AVG(mean) as mean, AVG(std) as std, AVG(meidan) as median from 
public.copernicus_nvdi where pixels_array is not null and tesela is not null
group by codigo, date;'''
satelital_data = pd.read_sql_query(query, con=conexion).drop_duplicates()
conexion.commit()
satelital_data

In [ ]:
subdatas_list=[]
for campo in satelital_data.codigo.unique():
    subdata=satelital_data[satelital_data.codigo==campo]
    subdata['date']=pd.to_datetime(subdata['date'],format='%Y-%m-%d')
    subdata['date2']=subdata['date']
    subdata=subdata.sort_values('date').set_index('date').resample('1D').ffill().reset_index()
    subdata['diff']=(subdata['date']-subdata['date2']).dt.days
    subdatas_list.append(subdata[subdata['diff']<21].drop(columns=['diff','date2']))
satelital_data2=pd.concat(subdatas_list)
satelital_data2['dia']=satelital_data2.date.dt.dayofyear
satelital_data2['anio']=satelital_data2.date.dt.year
satelital_data2=satelital_data2.drop(columns=['date'])
satelital_data2

In [ ]:
conexion.close()

In [ ]:
datos_total2=pd.merge(datos_total, satelital_data2, left_on=['codigo','anio','dia'], right_on=['codigo','anio','dia'])
# datos_total2=datos_total2.drop(columns=['codigo'])
datos_total2

In [ ]:
datos_total2.columns[400:500]

In [ ]:
def objective(trial,train,vali,estado_fen):
    variedades=[col for col in train.columns if 'variedad_' in col]
    variables_basic=['phenologystageid','dia', 'min', 'max', 'mean', 'std', 'median']+variedades
    medidas=variables_basic
    
    altitud=trial.suggest_categorical('Altitud',[True,False])
    latitud=trial.suggest_categorical('Latitud',[True,False])
    longitud=trial.suggest_categorical('Longitud',[True,False])
    if(longitud):
        medidas.append('longitude')
    if(latitud):
        medidas.append('latitude')
    if(altitud):
        medidas.append('altitud')
        
    if (trial.suggest_categorical('Acumuladas',[True,False])):
        
        inicio=trial.suggest_categorical('Inicio mediciones',['t0','1','2'])
        suffix=''
        name_vars=[]
        
        chilling=trial.suggest_categorical('Chilling',[True,False])
        winkler=trial.suggest_categorical('Winkler',[True,False])
        gdd=trial.suggest_categorical('gdd',[True,False])
        
        acumulativo=trial.suggest_categorical('Acumulativo',[True,False])
        if (acumulativo):
            suffix='_Cumm'
            if(trial.suggest_categorical('Precipitaciones',[True,False])):
                name_vars.append('precip__'+inicio+'_')
            if(trial.suggest_categorical('Radiacion',[True,False])):
                print('rad__'+inicio+'_')
                name_vars.append('rad__'+inicio+'_')
                
        if (winkler|gdd):
            tbase=trial.suggest_categorical('Tbase',['Tbase','TbaseMax'])
            temperatura_inicio=trial.suggest_categorical('Temperatura inicio',['10.0','4.5'])
        if (chilling):
            tbase_chill=trial.suggest_categorical('Tbase_chilling',['Tbase','Tbasemin','Utah'])
            name_vars.append('chillingDD_7.0_'+inicio+'_' + tbase_chill +'_sum')
        if (winkler):
            if acumulativo:
                name_vars.append('winkler_'+temperatura_inicio+'_'+inicio+'_' + tbase)
            else:
                name_vars.append('winkler_'+temperatura_inicio+'_' + tbase)
        if (gdd):
            name_vars.append('gdd_'+temperatura_inicio+'_'+inicio+'_' + tbase+'_sum')
       
        if len(name_vars)>0:
            semanas_list=[]
            for i in range(1,3):
                if(trial.suggest_categorical(str(i)+'_semanas_atras',[True,False])):
                    semanas_list.append(' '+str(i)+'_semanas_atras')
            if(trial.suggest_categorical('1_semanas_adelante',[True,False])):
                    semanas_list.append(' 1_semanas_adelante')
                    
            for name_var in name_vars:
                medidas.append(name_var+suffix)
                for semana in semanas_list:
                    medidas.append(name_var+suffix+semana)
        
    else:
        name_vars=[]    
        if(trial.suggest_categorical('Temperatura',[True,False])):
            if(trial.suggest_categorical('Temperatura_media',[True,False])):
                name_vars.append('tmed_mean')
            if(trial.suggest_categorical('Temperatura_min',[True,False])):
                name_vars.append('tmed_min')
            if(trial.suggest_categorical('Temperatura_max',[True,False])):
                name_vars.append('tmed_max')

        if(trial.suggest_categorical('Radiacion',[True,False])):
            if(trial.suggest_categorical('Radiacion_media',[True,False])):
                name_vars.append('rad_mean')
            if(trial.suggest_categorical('Radiacion_min',[True,False])):
                name_vars.append('rad_min')
            if(trial.suggest_categorical('Radiacion_max',[True,False])):
                name_vars.append('rad_max')

        if(trial.suggest_categorical('Viento',[True,False])):
            if(trial.suggest_categorical('Viento_norte',[True,False])):
                name_vars.append('wind_N')
            if(trial.suggest_categorical('Viento_noreste',[True,False])):
                name_vars.append('wind_NE')
            if(trial.suggest_categorical('Viento_este',[True,False])):
                name_vars.append('wind_E')
            if(trial.suggest_categorical('Viento_sureste',[True,False])):
                name_vars.append('wind_SE')
            if(trial.suggest_categorical('Viento_sur',[True,False])):
                name_vars.append('wind_S')
            if(trial.suggest_categorical('Viento_suroeste',[True,False])):
                name_vars.append('wind_SW')
            if(trial.suggest_categorical('Viento_oeste',[True,False])):
                name_vars.append('wind_W')
            if(trial.suggest_categorical('Viento_noroeste',[True,False])):
                name_vars.append('wind_NW')
        
        if len(name_vars)>0:
            dias_vars=[]
            for i in range(1,15):
                if(trial.suggest_categorical(str(i)+'_dias_atras',[True,False])):
                    dias_vars.append(' ' + str(i) + '_dias_atras')
            for i in range(1,8):
                if(trial.suggest_categorical(str(i)+'_dias_adelante',[True,False])):
                    dias_vars.append(' ' + str(i) + '_dias_adelante')

            for name_var in name_vars:
                medidas.append(name_var)
                for dia_var in dias_vars:
                    medidas.append(name_var+dia_var)

    medidas.append('dias_hasta')
    train=train[medidas].dropna()
    vali=vali[medidas].dropna()
    
    print(medidas)
    print(len(medidas))

    X_train=train.drop(['dias_hasta'], axis=1).values
    Y_train=train['dias_hasta'].values
    X_vali=vali.drop(['dias_hasta'], axis=1).values
    Y_vali=vali['dias_hasta'].values

    clear_session()

    inputA = Input(shape=(X_train.shape[1],), name='Entrada')

    y = Dense(int(trial.suggest_discrete_uniform('Neuronas capa 0',low=16,high=512,q=16)))(inputA)
    y = Activation(trial.suggest_categorical('Activacion capa 0',["selu","linear","tanh","softmax"]))(y)
    
    capas=trial.suggest_int('Numero de capas',low=3,high=8)
    
    for i in range(1,capas-2):
        y = Dense(int(trial.suggest_discrete_uniform('Neuronas capa {}'.format(i),low=16,high=512,q=64)))(y)
        y = Activation(trial.suggest_categorical('Activacion capa {}'.format(i),["selu","linear","tanh","softmax"]))(y)
        
    for i in range(capas-2,capas):
        y = Dense(int(trial.suggest_discrete_uniform('Neuronas capa {}'.format(i),low=16,high=256,q=16)))(y)
        y = Activation(trial.suggest_categorical('Activacion capa {}'.format(i),["selu","linear","tanh","softmax"]))(y)
        
    z = Dense(1, activation="selu")(y)

    model = Model(inputs=inputA, outputs=z)
    
    callback = [EarlyStopping(monitor='val_loss',patience=17, min_delta=1.7,restore_best_weights=False),
            ModelCheckpoint('/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/checkpointed_model.h5', monitor='val_loss',
                            save_best_only=True)]
    
    learning_rate=trial.suggest_categorical('Learning rate',[10**-3, 10**-2, 10**-1])
   
    choiceval = trial.suggest_categorical('Optimizador',['sgd','adam','rmsprop'])
    if choiceval == 'adam':
        optim = keras.optimizers.Adam(lr=learning_rate)
    elif choiceval == 'rmsprop':
        optim = keras.optimizers.RMSprop(lr=learning_rate)
    else:
        optim = keras.optimizers.SGD(lr=learning_rate)

    weights=None
    
    model.save('/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/checkpointed_model.h5')
    
    model.compile(loss='mse', optimizer=optim, metrics=['mse'])
    model.fit(X_train, Y_train, epochs=1000, verbose=0, 
              batch_size=trial.suggest_int('Batch size',low=1,high=16), 
              callbacks=callback, validation_split=0.2, class_weight=weights)
    
    model=load_model('/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/checkpointed_model.h5')
        
    
    preds = model.predict(X_vali)

    SCORE=r2_score(Y_vali,preds)
    print("R^2: ",SCORE)
    
    nombre='/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/model-estado'+ str(estado_fen) +'-'+str(SCORE)
     
    if SCORE>0.65:    
        model_json = model.to_json()
        with open(nombre+".json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights(nombre+".h5")
    
    print('#'*100)
    
    return SCORE

In [ ]:
datos_no_val=datos_total2[datos_total2.anio<=2020]
datos_no_val.phenologystageid=datos_no_val.phenologystageid.replace({9.0:0.0})

In [ ]:
random_state  = 17
direction     = 'maximize'
n_trials      = 5
n_jobs        = 1
timeout       = None
verbosity     = 0
trozos=20


for estado_fen in [3,5]:
    temp=datos_no_val.copy()

    datos_list=[]
    for campaña in temp.anio.unique():
        datos_camp=temp[temp['anio']==campaña]
        for id_terr in datos_camp.codigo.unique():
            datos_camp_terr=datos_camp[datos_camp['codigo']==id_terr]
            x=datos_camp_terr[datos_camp_terr['phenologystageid']>=estado_fen]['dia'].values
            momento=0
            if len(x>0):
                momento=np.min(x)
            datos_camp_terr['dias_hasta']=momento-datos_camp_terr['dia']
            datos_list.append(datos_camp_terr[datos_camp_terr['dias_hasta']>0])
    datos_final=pd.concat(datos_list)
    train=datos_final[datos_final.anio!=2018]
    vali=datos_final[datos_final.anio==2018]
    train=train.drop(columns=['codigo','anio'])
    print(len(train))
    vali=vali.drop(columns=['codigo','anio'])

    study_name = f'Estado-'+str(estado_fen)
    sampler    = optuna.samplers.TPESampler(seed=random_state)

    FILE = f'/data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/resumen_optuna-estado'+str(estado_fen)+'-r2-dias.csv'

    study = optuna.create_study(study_name=study_name,direction=direction,sampler=sampler,
                                storage='sqlite:////data/proyectos/GRAPEVINE/Models/all_data/Intento1/Estado_feno_' + str(estado_fen) + '/optuna-aceituna-estado'+str(estado_fen)+'-r2-dias.db', 
                                load_if_exists=True)

    for j in range(trozos):
        study.optimize(func=lambda trial: objective(trial,train,vali,estado_fen),
                       n_trials=n_trials,timeout=timeout)

        result_df = study.trials_dataframe()
        result_df.to_csv(FILE)
        time.sleep(1)